<a href="https://colab.research.google.com/github/chia-yu0225/work/blob/main/%E8%82%A1%E5%83%B9%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 台積電和連發科股價分析比較

# 摘要

1. 中文字型設定
使用 Taipei Sans TC Beta 解決圖表中文字亂碼問題。

2. 資料擷取與清洗
抓取台積電（2330）與聯發科（2454）在2025 年 5 月的每日股價。

 處理欄位格式（日期、收盤價、成交股數、漲跌價差）。
3. 資料視覺化分析
  收盤價折線圖比較

  比較兩檔股票的股價走勢。

  成交股數長條圖比較

  對比兩檔股票每日的成交量。

  雙軸圖（收盤價 + 成交量）

  同一檔股票的價格與成交量一起觀察。

  雙軸圖（成交量紅漲綠跌）

  成交量用顏色區分漲跌（紅色＝漲，綠色＝跌）增加可視辨識度。

# 程式主體

In [ ]:
# 字型下載與中文設定（只需執行一次）
!wget -O taipei_sans_tc_beta.ttf https://drive.google.com/uc?id=1eGAsTN1HBpJAkeVM57_C7ccp7hbgSz3_&export=download

import matplotlib
import matplotlib.pyplot as plt
matplotlib.font_manager.fontManager.addfont('taipei_sans_tc_beta.ttf')
plt.rcParams['font.sans-serif'] = ['Taipei Sans TC Beta']
plt.rcParams['axes.unicode_minus'] = False

# 載入 pandas 等模組
import pandas as pd

# 資料抓取函式
def get_stock_data(date, stock_no):
    url = f'https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY?date={date}&stockNo={stock_no}&response=html'
    df = pd.read_html(url, header=1)[0]
    df['股票代號'] = stock_no
    return df

# 設定查詢條件（2025 年 5 月，台積電+聯發科）
stock_list = ['2330', '2454']  # 台積電、聯發科
date = '20250501'
df_all = pd.DataFrame()

# 抓取並整合資料
for stock in stock_list:
    df_all = pd.concat([df_all, get_stock_data(date, stock)], ignore_index=True)

# 清洗資料
df_all['日期'] = pd.to_datetime(df_all['日期'], errors='coerce')
df_all['收盤價'] = pd.to_numeric(df_all['收盤價'], errors='coerce')
df_all['成交股數'] = pd.to_numeric(df_all['成交股數'].str.replace(',', ''), errors='coerce')
df_all['漲跌價差'] = pd.to_numeric(df_all['漲跌價差'], errors='coerce')
df_all = df_all.dropna(subset=['日期', '收盤價'])

# 顏色標示：漲紅跌綠
df_all['color'] = df_all['漲跌價差'].apply(lambda x: 'green' if x < 0 else 'red')

# 收盤價比較折線圖
plt.figure(figsize=(12,4))
for stock in stock_list:
    df = df_all[df_all['股票代號'] == stock]
    plt.plot(df['日期'], df['收盤價'], label=f'股票 {stock}')
plt.legend()
plt.title('收盤價比較')
plt.xlabel('日期')
plt.ylabel('收盤價')
plt.grid(True)
plt.show()

# 成交股數比較長條圖
plt.figure(figsize=(12,4))
for stock in stock_list:
    df = df_all[df_all['股票代號'] == stock]
    plt.bar(df['日期'], df['成交股數'], alpha=0.5, label=f'股票 {stock}')
plt.legend()
plt.title('成交股數比較')
plt.xlabel('日期')
plt.ylabel('成交股數')
plt.show()

# 雙軸圖：收盤價+成交量
for stock in stock_list:
    df = df_all[df_all['股票代號'] == stock]
    fig, ax1 = plt.subplots(figsize=(12,4))
    ax1.plot(df['日期'], df['收盤價'], color='blue')
    ax1.set_ylabel('收盤價', color='blue')

    ax2 = ax1.twinx()
    ax2.bar(df['日期'], df['成交股數'], color='gray', alpha=0.5)
    ax2.set_ylabel('成交股數', color='gray')

    plt.title(f'股票 {stock}：收盤價與成交量')
    plt.show()

# 雙軸圖：成交量以漲跌上色
for stock in stock_list:
    df = df_all[df_all['股票代號'] == stock]
    fig, ax1 = plt.subplots(figsize=(12,4))
    ax1.plot(df['日期'], df['收盤價'], color='blue')
    ax1.set_ylabel('收盤價', color='blue')

    ax2 = ax1.twinx()
    ax2.bar(df['日期'], df['成交股數'], color=df['color'], alpha=0.6)
    ax2.set_ylabel('成交股數', color='gray')

    plt.title(f'股票 {stock}：收盤價與成交量（紅漲綠跌）')
    plt.show()


# 未來展望

可以加入輸入股票代號」、「選擇月份」的互動功能